# Machine Translation Project
A deep neural network that functions as part of an end-to-end machine translation pipeline


In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [ ]:
import collections 

import helper
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam 
from keras.losses import sparse_categorical_crossentropy

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
Hello world 